In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/students-drop-out-prediction/train.csv
/kaggle/input/students-drop-out-prediction/test.csv


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

from xgboost import XGBClassifier

import seaborn as sns
sns.set(rc={'figure.figsize':(20, 20)})

In [6]:
train_df = pd.read_csv("../input/students-drop-out-prediction/train.csv")
test_df = pd.read_csv("../input/students-drop-out-prediction/test.csv")

In [7]:
all_col_names = list(set(train_df.columns) - set(["label", "id"])) 
binary_col_names = ["v_1", "v_26", "v_11", "v_14", "v_30", "v_28", "v_9", "v_27"]
non_binary_col_names = list(set(all_col_names) - set(binary_col_names))

In [8]:
transformer = ColumnTransformer(
    [
        ("drop_id", "drop", ["id"]),
        ("scale_non_binary", StandardScaler(), non_binary_col_names)
    ],
    remainder="passthrough"
)
transform_train_df = transformer.fit_transform(train_df)
feature_names = [name.split("__")[1] for name in transformer.get_feature_names_out()]
transform_train_df = pd.DataFrame(data=transform_train_df, columns=feature_names)
transform_train_df = transform_train_df.astype(dict([(col, "int") for col in binary_col_names]))

In [9]:
transform_train_df1 = transform_train_df.copy()
y_train_com = transform_train_df1.pop("label")
X_train_com = transform_train_df1
X_train, X_valid, y_train, y_valid = train_test_split(X_train_com, y_train_com, test_size=0.3, random_state=1)

In [34]:
tree_model = DecisionTreeClassifier(
    class_weight="balanced",
    max_features=None,
    ccp_alpha=0.0001,
    max_depth=10,
    min_samples_leaf=10,
    min_samples_split=5
)
model = AdaBoostClassifier(tree_model, random_state=1, n_estimators=50, learning_rate=1)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.75      0.67      0.71       375
         1.0       0.72      0.90      0.80       540
         2.0       0.54      0.31      0.40       224

    accuracy                           0.71      1139
   macro avg       0.67      0.63      0.63      1139
weighted avg       0.69      0.71      0.69      1139



In [37]:
model = GradientBoostingClassifier(
    random_state=1,
    min_samples_leaf=10,
    min_samples_split=5,
    max_features=None,
    ccp_alpha=0.0001,
    max_depth=10,
)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.77      0.70      0.73       375
         1.0       0.75      0.94      0.84       540
         2.0       0.56      0.32      0.41       224

    accuracy                           0.74      1139
   macro avg       0.70      0.65      0.66      1139
weighted avg       0.72      0.74      0.72      1139



In [ ]:
param_grid = {
    "subsample": [0.75, 1.0],
    "n_estimators": [100, 200],
    "learning_rate": [0.05, 0.1]
}
model = GradientBoostingClassifier(
    random_state=1,
    min_samples_leaf=10,
    min_samples_split=5,
    max_features=None,
    ccp_alpha=0.0001,
    max_depth=10,
)
search = GridSearchCV(model, param_grid, scoring="f1_macro", refit=True, cv=3)
search.fit(X_train_com, y_train_com)
print(search.best_params_)
print(search.best_score_)

In [42]:
model = GradientBoostingClassifier(
    random_state=1,
    min_samples_leaf=10,
    min_samples_split=5,
    max_features=None,
    ccp_alpha=0.0001,
    max_depth=10,
    subsample=0.75,
)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.78      0.72      0.75       375
         1.0       0.75      0.93      0.83       540
         2.0       0.59      0.33      0.42       224

    accuracy                           0.74      1139
   macro avg       0.71      0.66      0.67      1139
weighted avg       0.73      0.74      0.72      1139



In [10]:
model = XGBClassifier(random_state=1)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.78      0.71      0.74       375
         1.0       0.77      0.93      0.84       540
         2.0       0.60      0.38      0.47       224

    accuracy                           0.75      1139
   macro avg       0.72      0.67      0.68      1139
weighted avg       0.74      0.75      0.73      1139



In [11]:
# Tuning of parameters related to DecisionTree
param_grid = {
    "min_child_weight": [0.5, 1, 2],
    "max_depth": [5, 10, 15],
    "subsample": [0.5, 0.75, 1],
}
model = XGBClassifier(random_state=1)
search = GridSearchCV(
    model,
    param_grid,
    scoring="f1_macro",
    refit=True,
    cv=3
)
search.fit(X_train_com, y_train_com)
print(search.best_params_)
print(search.best_score_)

{'max_depth': 5, 'min_child_weight': 0.5, 'subsample': 0.75}
0.6738833945206905


In [12]:
model = XGBClassifier(random_state=1, max_depth=5, min_child_weight=0.5, subsample=0.75)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.78      0.73      0.75       375
         1.0       0.77      0.93      0.84       540
         2.0       0.58      0.35      0.44       224

    accuracy                           0.75      1139
   macro avg       0.71      0.67      0.68      1139
weighted avg       0.74      0.75      0.73      1139



In [13]:
# Tuning of parameters related to boosting
param_grid = {
     "lambda": [1, 3, 5],
     "alpha": [0, 1, 3],
     "eta": [0.01, 0.05, 0.1],
}
model = XGBClassifier(
    random_state=1,
    max_depth=5,
    min_child_weight=0.5,
    subsample=0.75
)
search = GridSearchCV(
    model,
    param_grid,
    scoring="f1_macro",
    refit=True,
    cv=3
)
search.fit(X_train_com, y_train_com)
print(search.best_params_)
print(search.best_score_)

{'alpha': 1, 'eta': 0.1, 'lambda': 3}
0.6742813892687317


In [15]:
# Tuning of parameters related to boosting
param_grid = {
     "eta": [0.1, 0.2, 0.3, 0.4, 0.5],
}
kwargs = {"lambda": 3}
model = XGBClassifier(
    random_state=1,
    max_depth=5,
    min_child_weight=0.5,
    subsample=0.75,
    alpha=1,
    **kwargs
)
search = GridSearchCV(
    model,
    param_grid,
    scoring="f1_macro",
    refit=True,
    cv=3
)
search.fit(X_train_com, y_train_com)
print(search.best_params_)
print(search.best_score_)

{'eta': 0.1}
0.6742813892687317


In [16]:
model = XGBClassifier(random_state=1, max_depth=5, min_child_weight=0.5, subsample=0.75, eta=0.1)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

         0.0       0.79      0.73      0.76       375
         1.0       0.76      0.94      0.84       540
         2.0       0.60      0.34      0.44       224

    accuracy                           0.75      1139
   macro avg       0.72      0.67      0.68      1139
weighted avg       0.74      0.75      0.73      1139

